# Scope

In [1]:
# When Python compiles the body of the function, it decides that b is a local variable because it is assigned within the function

b = 'b'

def f1(a):
    print(a)
    print(b)
    b = 1

try:
    f1('a')
except Exception as e:
    print(e)


a
local variable 'b' referenced before assignment


In [2]:
# to fix this we can use global declaration

b = 'b'

def f2(a):
    global b
    print(a)
    print(b)
    b = 1

f2('a')

a
b


# Closure

Closures is what we get when functions capture variables defined outside of their bodies.  
  
**Closure** is a function *f* with an extended scope that encompases variables referenced in the body of *f* that are not global variables or local variables of *f*  
  
Closures allow us to preserve state while using functional approach (see [running average](../tasks/running_average.ipynb))

# Variable Lookup Logic

- if there is a *global x* declaration, *x* comes from and is assigned to module global variable *x* (python does not have a program global scope, only module global scopes)
- if there is a *nonlocal x* declaration, *x* comes from and is assigned to local variable *x* of the nearest surrounding function where *x* is defined
- if *x* is a parameter or is assigned a value in the function body, the *x* is the local variable
- if *x* is referenced but is not assigned and is not a parameter:
    - *x* will be looked up in the local scopes of the surrounding function bodies (nonlocal scopes)
    - if not found in surrounding scopes, it will be read from the module global scope
    - if not found in the global scope, it will be read from \_\_builtins__.__dict__.

# Decorators

A decorator is a callable that takes another function as an argument, 
extending the behavior of that function without explicitly modifying that function

https://dev.to/apcelent/python-decorator-tutorial-with-example-529f#

```
@decorator
def target():
    ...
```
the same as:

```
target = decorator(target)
```

Decorator function is executed when decorated function is defined (at import time as soon as the module is imported).

## Function decorator

In [5]:
def handleException(func):
    def wrapper(*args):
        try:
            return func(*args)
        except TypeError:
            print('Type error')
        except ZeroDivisionError:
            print('Zero division error')
        except:
            print('Some error')
    return wrapper

@handleException
def causeError(n):
    return 1/n

# @handleException - shortcut for:
# causeError = handleException(causeError)

print(causeError(1))
causeError(0)

1.0
Zero division error


In [6]:
from functools import wraps

def my_decorator(func):
  @wraps(func)              # to preserve func name and docstring
  def wrapper(*args, **kwargs):
    # do something before
    result = func(*args, **kwargs)
    # do something after
    return result
  return wrapper

@my_decorator  
def func():
  return 'function output'
  
# equivalent to
func = my_decorator(func)

func()

'function output'

## Class decorator

In [7]:
from functools import update_wrapper

class Count():
  def __init__(self, func):
    update_wrapper(self, func)          # for correct docstring and name of the function
    self.func = func
    self.cnt = 0
  def __call__(self, *args, **kwargs):
    self.cnt += 1
    print(f"Current count: {self.cnt}")
    result = self.func(*args, **kwargs)
    return result

@Count
def fib(n):
  if n < 2:
    return n
  else:
    return fib(n-1) + fib(n-2)

fib(4)      # 9 calls bcz we are not caching values and calculate many of them several times during recursive execution

Current count: 1
Current count: 2
Current count: 3
Current count: 4
Current count: 5
Current count: 6
Current count: 7
Current count: 8
Current count: 9


3

In [8]:
# for caching we can use lru_cache from functools

from functools import wraps, lru_cache

@lru_cache(maxsize=None)
@Count
def fib(n):
  if n < 2:
    return n
  else:
    return fib(n-1) + fib(n-2)

fib(4)

Current count: 1
Current count: 2
Current count: 3
Current count: 4
Current count: 5


3